In [1]:
import rmgpy
import numpy as np
from rmgpy.molecule.molecule import *
from rmgpy.species import *
from rmgpy.chemkin import *
from rmgpy.data.rmg import RMGDatabase
from IPython.display import display
from rmgpy.data.thermo import ThermoLibrary
from rmgpy.rmg.react import react
from rmgpy.species import Species
from rmgpy.reaction import Reaction
from rmgpy.data.rmg import get_db
from rmgpy.molecule.group import Group
from rmgpy.kinetics.arrhenius import ArrheniusBM
from rmgpy import settings
import time
import matplotlib.pyplot as plt
import matplotlib

In [2]:
settings

{'database.directory': '/home/khalil.nor/Code/RMG-database/input',
 'test_data.directory': '/home/khalil.nor/Code/RMG-Py/rmgpy/test_data'}

In [ ]:
#these families just had training rxns added to them 

#dict_keys(['1,2_Insertion_carbene', 'R_Addition_MultipleBond', 'XY_Addition_MultipleBond', 'Singlet_Carbene_Intra_Disproportionation', '1,2_Insertion_CO', '1+2_Cycloaddition', '1,3_Insertion_CO2', '1,3_sigmatropic_rearrangement'])

In [3]:
thermo_libs = [
'C1_C2_Fluorine', #adding Siddha's as first most trusted because this is the thermo library that Franklin used
'NCSU_C2_C8_PFAS', #adding Westmoreland's thermo as the second most trusted
'primaryThermoLibrary',
'Fluorine',
'FFCM1(-)',
'halogens',
'CHOF_G4',
'CHOCl_G4',
'CHOBr_G4',
'CHOFCl_G4',
'CHOFBr_G4',
'CHOFClBr_G4',
'DFT_QCI_thermo',
'2-BTP_G4',
'thermo_DFT_CCSDTF12_BAC',
'SulfurHaynes'
]

kin_families = ['R_Recombination', '1,2_Insertion_carbene', 'F_Abstraction', 'R_Addition_MultipleBond', 'XY_Addition_MultipleBond', 'Singlet_Carbene_Intra_Disproportionation', 'Intra_R_Add_Endocyclic', 'R_Addition_COm', '1,2_Insertion_CO', '1+2_Cycloaddition', '1,3_Insertion_CO2', '1,3_sigmatropic_rearrangement']


In [4]:
database = RMGDatabase()
database.load(
            path = settings['database.directory'],
            thermo_libraries = thermo_libs,
            transport_libraries = [],
            reaction_libraries = [],
            seed_mechanisms = [],#['BurkeH2O2inN2','ERC-FoundationFuelv0.9'],
            kinetics_families = kin_families,
            kinetics_depositories = ['training'],
            #frequenciesLibraries = self.statmechLibraries,
            depository = False, # Don't bother loading the depository information, as we don't use it
        )


1 *1 R u0 {2,[D,T]}
2 *2 R u0 {1,[D,T]}
3 *3 R u0 p[1,2,3]

1 *2 [H,Cs,Cd,Cb,Ct,CO,O,Sis,Sid,N,S]              u0 {2,S}
2 *3 [H,F1s,Cl1s,Br1s,I1s,Cs,Cd,Cb,Ct,Sis,Sid,N,S] u0 {1,S}
3 *1 C                                             u0 p1 c-1 {4,T}
4 *4 [O,S]                                         u0 p1 c+1 {3,T}

1 *1 C2s                                             u0 p1 {2,S} {3,S}
2    [H,F1s,Cl1s,Br1s,I1s]                           u0 {1,S}
3    [H,F1s,Cl1s,Br1s,I1s,C]                         u0 {1,S}
4 *2 [H,F1s,Cl1s,Br1s,I1s,Cs,Cd,Cb,Ct,O,Sis,Sid,N,S] u0 {5,S}
5 *3 [H,F1s,Cl1s,Br1s,I1s,Cs,Cd,Cb,Ct,Sis,Sid,N,S]   u0 {4,S}

1 *1 Cdd       u0 {2,D} {3,D}
2 *2 [O2d,S2d] u0 {1,D}
3    [O2d,S2d] u0 {1,D}
4 *3 R         u0 {5,S}
5 *4 R         u0 {4,S}

1 *1 R u1
2 *2 R u0 {3,[D,T,B]}
3 *3 R u0 {2,[D,T,B]}

1 *1 C2tc u0 p1 c-1 {2,T}
2 *3 O4tc u0 p1 c+1 {1,T}
3 *2 R    u1

1 *1 R!H u0         {2,[D,T,B]}
2 *2 R!H u0         {1,[D,T,B]}
3 *3 R   u[1,2,3,4]

1 * R u1
2 * R u1

1 *2 C       

In [5]:
database.kinetics.families

{'1+2_Cycloaddition': <ReactionFamily "1+2_Cycloaddition">,
 '1,2_Insertion_CO': <ReactionFamily "1,2_Insertion_CO">,
 '1,2_Insertion_carbene': <ReactionFamily "1,2_Insertion_carbene">,
 '1,3_Insertion_CO2': <ReactionFamily "1,3_Insertion_CO2">,
 '1,3_sigmatropic_rearrangement': <ReactionFamily "1,3_sigmatropic_rearrangement">,
 'F_Abstraction': <ReactionFamily "F_Abstraction">,
 'Intra_R_Add_Endocyclic': <ReactionFamily "Intra_R_Add_Endocyclic">,
 'R_Addition_COm': <ReactionFamily "R_Addition_COm">,
 'R_Addition_MultipleBond': <ReactionFamily "R_Addition_MultipleBond">,
 'R_Recombination': <ReactionFamily "R_Recombination">,
 'Singlet_Carbene_Intra_Disproportionation': <ReactionFamily "Singlet_Carbene_Intra_Disproportionation">,
 'XY_Addition_MultipleBond': <ReactionFamily "XY_Addition_MultipleBond">}

In [6]:
# 'Intra_R_Add_Endocyclic' 
family_to_train = "Intra_R_Add_Endocyclic"
family = database.kinetics.families[family_to_train]

In [7]:
family.clean_tree()

In [8]:
start = time.time()
family.generate_tree(thermo_database=database.thermo,
                     nprocs=1,
                     new_fraction_threshold_to_reopt_node=0.25,
                     max_batch_size=800,
                     extension_iter_max=2,
                     extension_iter_item_cap=100)

end = time.time()
print(end-start)

ERROR:root:getting reaction matches
ERROR:root:building tree with 800 rxns
ERROR:root:800.0
ERROR:root:iter_max achieved terminating early
ERROR:root:iter_max achieved terminating early
ERROR:root:iter_max achieved terminating early
ERROR:root:iter_max achieved terminating early
ERROR:root:iter_max achieved terminating early
ERROR:root:iter_max achieved terminating early
ERROR:root:iter_max achieved terminating early
ERROR:root:iter_max achieved terminating early
ERROR:root:iter_max achieved terminating early
ERROR:root:iter_max achieved terminating early
ERROR:root:iter_max achieved terminating early
ERROR:root:iter_max achieved terminating early
ERROR:root:iter_max achieved terminating early
ERROR:root:iter_max achieved terminating early
ERROR:root:Recursion item cap hit not splitting 3 reactions at iter 4 with 114 items
ERROR:root:Recursion item cap hit not splitting 2 reactions at iter 4 with 120 items
ERROR:root:iter_max achieved terminating early
ERROR:root:iter_max achieved term

671.0395753383636


In [9]:
len(family.groups.entries)

1029

In [10]:
family.groups.entries

{'RnCyclic': <Entry index=-1 label="RnCyclic">,
 'Root': <Entry index=0 label="Root">,
 'Root_1R-inRing': <Entry index=1 label="Root_1R-inRing">,
 'Root_N-1R-inRing': <Entry index=2 label="Root_N-1R-inRing">,
 'Root_1R-inRing_2R->C': <Entry index=3 label="Root_1R-inRing_2R->C">,
 'Root_1R-inRing_N-2R->C': <Entry index=4 label="Root_1R-inRing_N-2R->C">,
 'Root_N-1R-inRing_3R->O': <Entry index=5 label="Root_N-1R-inRing_3R->O">,
 'Root_N-1R-inRing_N-3R->O': <Entry index=6 label="Root_N-1R-inRing_N-3R->O">,
 'Root_1R-inRing_2R->C_Int-3R-1R': <Entry index=7 label="Root_1R-inRing_2R->C_Int-3R-1R">,
 'Root_1R-inRing_2R->C_Ext-3R-R_Ext-1R-R_Ext-3R-R_Int-5R!H-4R!H': <Entry index=8 label="Root_1R-inRing_2R->C_Ext-3R-R_Ext-1R-R_Ext-3R-R_Int-5R!H-4R!H">,
 'Root_1R-inRing_2R->C_Ext-3R-R_4R!H-inRing': <Entry index=9 label="Root_1R-inRing_2R->C_Ext-3R-R_4R!H-inRing">,
 'Root_1R-inRing_2R->C_Ext-3R-R_N-4R!H-inRing': <Entry index=10 label="Root_1R-inRing_2R->C_Ext-3R-R_N-4R!H-inRing">,
 'Root_N-1R-inRi

In [11]:
start = time.time()
family.check_tree()
end = time.time()
print(end-start)

0.9641017913818359


In [12]:
start = time.time()
family.regularize(thermo_database=database.thermo)
end = time.time()
print(end-start)

150.05800032615662


In [13]:
start = time.time()
templateRxnMap = family.get_reaction_matches(thermo_database=database.thermo,remove_degeneracy=True,
                                             get_reverse=True,exact_matches_only=False,fix_labels=True)
end = time.time()
print(end-start)

148.05686378479004


In [14]:
len(templateRxnMap)

1029

In [15]:
family.clean_tree_rules()

In [16]:
start = time.time()
family.make_bm_rules_from_template_rxn_map(templateRxnMap)#,nprocs=6)
end = time.time()
print(end-start)

/home/khalil.nor/Code/RMG-Py/rmgpy/data/kinetics/family.py:3640: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  for rxns, label in rxnlists])
/work/westgroup/nora/miniconda/envs/rmg_env/lib/python3.7/site-packages/scipy/optimize/minpack.py:834: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
/work/westgroup/nora/miniconda/envs/rmg_env/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3441: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/work/westgroup/nora/miniconda/envs/rmg_env/lib/python3.7/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


758.7483961582184


In [17]:
start = time.time()
family.check_tree()
end = time.time()
print(end-start)

0.9596526622772217


In [18]:
start = time.time()
errors,uncertainties = family.cross_validate(iters=0,random_state=5,folds=0,ascend=False)
end = time.time()
print(end-start)

/work/westgroup/nora/miniconda/envs/rmg_env/lib/python3.7/site-packages/scipy/optimize/minpack.py:834: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


154.21562552452087


In [19]:
save_path = os.path.join(settings['database.directory'], 'kinetics', 'families', family.name)
print(save_path)

/home/khalil.nor/Code/RMG-database/input/kinetics/families/Intra_R_Add_Endocyclic


In [20]:
family.save(save_path)

In [ ]:
#completed on May 14th, 2024 at 11:53 am